This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# PC calibration: "moving-screen" technique

The gnomonic projection (pattern) center (PC) of an EBSD detector can be
estimated by the "moving-screen" technique
<cite data-cite="hjelen1991electron">Hjelen et al.</cite>, which we will test in
this tutorial.

The technique relies on the assumption that the beam normal, shown in the
[top figure (d) in the reference frames tutorial](reference_frames.ipynb#sample-detector-geometry),
is normal to the detector screen as well as the incoming electron beam, and will
therefore intersect the screen at a position independent of the detector
distance (DD). To find this position, we need two EBSD patterns acquired with a
stationary beam but with a known difference $\Delta z$ in DD, say 5 mm.

First, the goal is to find the pattern position which does not shift between the
two camera positions, ($PC_x$, $PC_y$). This point can be estimated in fractions
of screen width and height, respectively, by selecting the same pattern features
in both patterns. The two points of each pattern feature can then be used to
form a straight line, and two or more such lines should intersect at ($PC_x$,
$PC_y$).

Second, the DD ($PC_z$) can be estimated from the same points. After finding
the distances $L_{in}$ and $L_{out}$ between two points (features) in both
patterns (in = operating position, out = 5 mm from operating position), the DD
can be found from the relation

$$
\mathrm{DD} = \frac{\Delta z}{L_{out}/L_{in} - 1},
$$

where DD is given in the same unit as the known camera distance difference. If
also the detector pixel size $\delta$ is known (e.g. 46 mm / 508 px), $PC_z$ can
be given in the fraction of the detector screen height

$$
PC_z = \frac{\mathrm{DD}}{N_r \delta b},
$$

where $N_r$ is the number of detector rows and $b$ is the binning factor.

Let's first import necessary libraries

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

from diffsims.crystallography import ReciprocalLatticeVector
from orix.crystal_map import Phase
from orix.quaternion import Rotation
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

We will find an estimate of the PC from two single crystal Silicon EBSD
patterns, which are included in the
[kikuchipy.data](../reference/generated/kikuchipy.data.rst) module

In [ ]:
s_in = kp.data.silicon_ebsd_moving_screen_in(allow_download=True)
s_in.remove_static_background()
s_in.remove_dynamic_background()

s_out5mm = kp.data.silicon_ebsd_moving_screen_out5mm(allow_download=True)
s_out5mm.remove_static_background()
s_out5mm.remove_dynamic_background()

As a first approximation, we can find the detector pixel positions of the same
features in both patterns by plotting them and noting the upper right
coordianates provided by Matplotlib when plotting with an interactive backend
(e.g. qt5 or notebook) and hovering over image pixels

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(20, 10))
ax[0].imshow(s_in.data, cmap="gray")
_ = ax[1].imshow(s_out5mm.data, cmap="gray")

For this example we choose the positions of three zone axes. The PC calibration
is performed by creating an instance of the
[PCCalibrationMovingScreen](../reference/generated/kikuchipy.detectors.PCCalibrationMovingScreen.rst)
class

In [ ]:
cal = kp.detectors.PCCalibrationMovingScreen(
    pattern_in=s_in.data,
    pattern_out=s_out5mm.data,
    points_in=[(109, 131), (390, 139), (246, 232)],
    points_out=[(77, 146), (424, 156), (246, 269)],
    delta_z=5,
    px_size=None,  # Default
    convention="tsl",  # Default
)
cal

We see that ($PC_x$, $PC_y$) = (0.5123, 0.8606), while DD = 21.7 mm. To get
$PC_z$ in fractions of detector height, we have to provide the detector pixel
size $\delta$ upon initialization, or set it directly and recalculate the PC

In [ ]:
cal.px_size = 46 / 508  # mm/px
cal

We can visualize the estimation by using the (opinionated) convenience method
[PCCalibrationMovingScreen.plot()](../reference/generated/kikuchipy.detectors.PCCalibrationMovingScreen.plot.rst)

In [ ]:
cal.plot()

As expected, the three lines in the right figure meet at a more or less the same
position. We can replot the three images and zoom in on the PC to see how close
they are to each other. We will use two standard deviations of all $PC_x$
estimates as the axis limits (scaled with pattern shape)

In [ ]:
# PCy defined from top to bottom, otherwise "tsl", defined from bottom to top
cal.convention = "bruker"
pcx, pcy, _ = cal.pc
two_std = 2 * np.std(cal.pcx_all, axis=0)

fig, ax = cal.plot(return_fig_ax=True)
ax[2].set_xlim([cal.ncols * (pcx - two_std), cal.ncols * (pcx + two_std)])
_ = ax[2].set_ylim([cal.nrows * (pcy - two_std), cal.nrows * (pcy + two_std)])

Finally, we can use this PC estimate along with the orientation of the Si
crystal, as determined by Hough indexing with a commercial software, to see how
good the estimate is, by performing a
[geometrical EBSD simulation](geometrical_ebsd_simulations.rst)
of positions of Kikuchi band centres and zone axes from the five $\{hkl\}$
families $\{111\}$, $\{200\}$, $\{220\}$, $\{222\}$, and $\{311\}$

In [ ]:
phase = Phase(space_group=227)

# Specify which reflectors to use in simulation
ref = ReciprocalLatticeVector(
    phase=phase, hkl=[[1, 1, 1], [2, 0, 0], [2, 2, 0], [2, 2, 2], [3, 1, 1]]
)
ref = ref.symmetrise()  # Symmetrise to get all symmetrically equivalent planes

# Create simulator
simulator = kp.simulations.KikuchiPatternSimulator(ref)

# Specify detector and crystal orientation to simulate a pattern for
detector = kp.detectors.EBSDDetector(
    shape=cal.shape, pc=cal.pc, sample_tilt=70, convention=cal.convention
)
r = Rotation.from_euler(np.deg2rad([133.3, 88.7, 177.8]))

sim = simulator.on_detector(detector, r)

In [ ]:
# del s_in.metadata.Markers  # Uncomment this if we want to re-add markers
s_in.add_marker(marker=sim.as_markers(), plot_marker=False, permanent=True)
s_in.plot(navigator=None, colorbar=False, axes_off=True, title="")

The PC is not perfect, but the estimate might be good enough for a further PC
and/or orientation refinement.